### Dependencies

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import open3d as o3d
import time
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle >180.0:
        angle = 360-angle

    return angle

In [27]:
vid_to_analyze = 'USER VIDEO PATH'

In [29]:
columns = pd.MultiIndex.from_tuples([
    ('Left Hip', 'x'), ('Left Hip', 'y'), ('Left Hip', 'Angle'),
    ('Right Hip', 'x'), ('Right Hip', 'y'), ('Right Hip', 'Angle'),
    ('Left Knee', 'x'), ('Left Knee', 'y'), ('Left Knee', 'Angle'),
    ('Right Knee', 'x'), ('Right Knee', 'y'), ('Right Knee', 'Angle'),
    ('Left Ankle', 'x'), ('Left Ankle', 'y'), ('Left Ankle', 'Angle'),
    ('Right Ankle', 'x'), ('Right Ankle', 'y'), ('Right Ankle', 'Angle'),
    ('Left Foot Middle', 'x'), ('Left Foot Middle', 'y'),
    ('Right Foot Middle', 'x'), ('Right Foot Middle', 'y')
])
motion_df = pd.DataFrame(columns=columns)
motion_df.index.name = 'Frame'

frame_num = 0

cap = cv2.VideoCapture(vid_to_analyze)
with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        #Recolor Image to RGB 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make Detection
        results = pose.process(image)

        #Recolor image back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Image Dimensions:

        image_height = image.shape[0]
        image_width = image.shape[1]

        #Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].z]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].z]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]
            left_foot = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].z]
            right_foot = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].z]
            left_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].z]
            right_heel = [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].z]
            
            #Calculate Angle
            left_shoulder_angle = calculate_angle(left_elbow, left_shoulder, left_hip)
            right_shoulder_angle = calculate_angle(right_elbow, right_shoulder, right_hip)
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            left_hip_angle = calculate_angle(left_knee, left_hip, left_shoulder)
            right_hip_angle = calculate_angle(right_knee, right_hip, right_shoulder)
            left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            left_ankle_angle = calculate_angle(left_knee, left_ankle, left_foot)
            right_ankle_angle = calculate_angle(right_knee, right_ankle, right_foot)
            left_shoulder_secondary_angle = calculate_angle(left_elbow, left_shoulder, right_shoulder)
            right_shoulder_secondary_angle = calculate_angle(right_elbow, right_shoulder, left_shoulder)
            left_hip_secondary_angle = calculate_angle(left_knee, left_hip, right_hip)
            right_hip_secondary_angle = calculate_angle(right_knee, right_hip, left_hip)

            #Calculate Foot Mid Point

            left_foot_middle = [(left_heel[0] + left_foot[0])/2, (left_heel[1] + left_foot[1])/2, (left_heel[2] + left_foot[2])/2]
            right_foot_middle = [(right_heel[0] + right_foot[0])/2, (right_heel[1] + right_foot[1])/2, (right_heel[2] + right_foot[2])/2]

            #Construct Motion Data
            row_data = {
                ('Left Hip', 'x'): left_hip[0],
                ('Left Hip', 'y'): left_hip[1],
                ('Left Hip', 'Angle'): left_hip_angle,
                ('Left Hip', 'Secondary Angle'): left_hip_secondary_angle,
                ('Right Hip', 'x'): right_hip[0],
                ('Right Hip', 'y'): right_hip[1],
                ('Right Hip', 'Angle'): right_hip_angle,
                ('Right Hip', 'Secondary Angle'): right_hip_secondary_angle,
                ('Left Knee', 'x'): left_knee[0],
                ('Left Knee', 'y'): left_knee[1],
                ('Left Knee', 'Angle'): left_knee_angle,
                ('Right Knee', 'x'): right_knee[0],
                ('Right Knee', 'y'): right_knee[1],
                ('Right Knee', 'Angle'): right_knee_angle,
                ('Left Ankle', 'x'): left_ankle[0],
                ('Left Ankle', 'y'): left_ankle[1],
                ('Left Ankle', 'Angle'): left_ankle_angle,
                ('Right Ankle', 'x'): right_ankle[0],
                ('Right Ankle', 'y'): right_ankle[1],
                ('Right Ankle', 'Angle'): right_ankle_angle,
                ('Left Foot Middle', 'x'): left_foot_middle[0],
                ('Left Foot Middle', 'y'): left_foot_middle[1],
                ('Right Foot Middle', 'x'): right_foot_middle[0],
                ('Right Foot Middle', 'y'): right_foot_middle[1]
            }

            motion_df.loc[frame_num] = row_data

            frame_num += 1



        except Exception as e:
            pass

        #Render Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )


        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)


I0000 00:00:1740291482.511690  182203 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4
W0000 00:00:1740291482.562711  218836 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740291482.570473  218836 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [30]:
#Use image dimensions to un-normalize landmark data
motion_df[('Left Hip', 'x')] = motion_df[('Left Hip', 'x')]*image_width
motion_df[('Left Hip', 'y')] = motion_df[('Left Hip', 'y')]*image_height
motion_df[('Right Hip', 'x')] = motion_df[('Right Hip', 'x')]*image_width
motion_df[('Right Hip', 'y')] = motion_df[('Right Hip', 'y')]*image_height
motion_df[('Left Knee', 'x')] = motion_df[('Left Knee', 'x')]*image_width
motion_df[('Left Knee', 'y')] = motion_df[('Left Knee', 'y')]*image_height
motion_df[('Right Knee', 'x')] = motion_df[('Right Knee', 'x')]*image_width
motion_df[('Right Knee', 'y')] = motion_df[('Right Knee', 'y')]*image_height
motion_df[('Left Ankle', 'x')] = motion_df[('Left Ankle', 'x')]*image_width
motion_df[('Left Ankle', 'y')] = motion_df[('Left Ankle', 'y')]*image_height
motion_df[('Right Ankle', 'x')] = motion_df[('Right Ankle', 'x')]*image_width
motion_df[('Right Ankle', 'y')] = motion_df[('Right Ankle', 'y')]*image_height
motion_df[('Left Foot Middle', 'x')] = motion_df[('Left Foot Middle', 'x')]*image_width
motion_df[('Left Foot Middle', 'y')] = motion_df[('Left Foot Middle', 'y')]*image_height
motion_df[('Right Foot Middle', 'x')] = motion_df[('Right Foot Middle', 'x')]*image_width
motion_df[('Right Foot Middle', 'y')] = motion_df[('Right Foot Middle', 'y')]*image_height


#Set Velocity and Acceleration Columns for Each Landmark
for landmark in ['Left Hip', 'Right Hip', 'Left Knee', 'Right Knee', 'Left Ankle', 'Right Ankle']:
    angular_velocity = motion_df[(landmark, 'Angle')].diff()
    angular_acceleration = np.deg2rad(angular_velocity.diff())
    motion_df[(landmark, 'Angular Acceleration')] = angular_acceleration
for landmark in ['Left Foot Middle', 'Right Foot Middle']:
    if landmark == 'Left Foot Middle':
        name = 'cop_L'
    else:
        name = 'cop_R'
    y_col = motion_df[(landmark, 'y')]
    x_col = motion_df[(landmark, 'x')]
    vel_x = x_col.diff()
    vel_y = y_col.diff()
    acc_x = vel_x.diff()
    acc_y = vel_y.diff()
    
    motion_df[(landmark, 'Vel_x')] = vel_x
    motion_df[(landmark, 'Vel_y')] = vel_y
    motion_df[(landmark, 'Acc_x')] = acc_x
    motion_df[(landmark, 'Acc_y')] = acc_y




#Interpolate Missing Values
cleaned_motion_df = motion_df.interpolate(method='linear', limit_direction='forward')
cleaned_motion_df.fillna(0, inplace=True)



In [31]:
cleaned_motion_df

Left Hip                           Right Hip                          \
                x           y       Angle           x           y       Angle   
Frame                                                                           
0      484.622067  516.190932  172.722274  386.081599  521.689804  173.772676   
1      484.232980  515.581899  172.764216  385.352599  520.257279  174.035576   
2      484.190542  514.966686  172.851308  385.114314  519.018935  174.115627   
3      484.195467  514.734814  172.930039  385.114387  518.709784  174.141637   
4      484.202129  514.752452  172.919586  385.130681  518.733505  174.218508   
...           ...         ...         ...         ...         ...         ...   
287    623.151188  628.195925  174.114133  419.134142  631.602609  175.384264   
288    624.581911  628.433590  174.311067  421.792474  631.656811  176.567589   
289    633.392147  628.239698  174.465226  430.509530  630.002553  177.978512   
290    630.868081  625.784190  176.100620  470.798074  626.862438  178.145208   
291    629.025097  624.993432  176.897021  469.363537  624.929960  179.948052   

        Left Knee                          Right Knee  ...  \
                x           y       Angle           x  ...   
Frame                                                  ...   
0      500.704007  684.669213  178.475596  392.666211  ...   
1      499.247358  684.077175  179.169102  392.671714  ...   
2      498.423319  683.551826  179.572981  392.672463  ...   
3      497.993581  683.450761  179.805143  392.729119  ...   
4      497.956020  683.432415  179.872417  392.996999  ...   
...           ...         ...         ...         ...  ...   
287    622.337337  988.081512  178.260233  436.109982  ...   
288    620.404649  991.358099  178.993872  447.493626  ...   
289    622.271483  987.026117  179.031938  463.359901  ...   
290    609.861218  988.210258  179.143458  503.214024  ...   
291    607.022561  987.524107  179.121280  508.993980  ...   

                Left Ankle          Right Ankle Left Foot Middle             \
      Angular Acceleration Angular Acceleration            Vel_x      Vel_y   
Frame                                                                         
0                 0.000000             0.000000         0.000000   0.000000   
1                 0.000000             0.000000        -0.184091   0.035598   
2                -0.008156            -0.000298        -0.112733   0.037433   
3                -0.003729             0.000078        -0.003766   0.057163   
4                -0.001116             0.002030         0.036379   0.116193   
...                    ...                  ...              ...        ...   
287              -0.037522            -0.104208         1.792314 -26.838462   
288               0.037856             0.102482         2.514482  -0.506358   
289              -0.004035            -0.016893        12.965458 -17.723629   
290               0.102885             0.025588       -26.828243 -16.629031   
291              -0.193132            -0.093282       -11.072312   7.323911   

                            Right Foot Middle                                   
           Acc_x      Acc_y             Vel_x      Vel_y      Acc_x      Acc_y  
Frame                                                                           
0       0.000000   0.000000          0.000000   0.000000   0.000000   0.000000  
1       0.000000   0.000000          0.026143   1.383247   0.000000   0.000000  
2       0.071357   0.001835         -0.014617   0.747854  -0.040760  -0.635394  
3       0.108967   0.019730         -0.076680   0.432780  -0.062064  -0.315074  
4       0.040145   0.059030         -0.048871   0.245551   0.027809  -0.187229  
...          ...        ...               ...        ...        ...        ...  
287     1.813412 -75.424962         14.401177  -0.134218  20.462074 -71.745143  
288     0.722169  26.332104          1.895319  16.295707 -12.505859  16.429925  
289    

In [32]:
import joblib

model = joblib.load('xgb_prediction_model.pkl')

features = np.column_stack([
    cleaned_motion_df[('Left Foot Middle', 'x')].values,
    cleaned_motion_df[('Left Foot Middle', 'y')].values,
    cleaned_motion_df[('Right Foot Middle', 'x')].values,
    cleaned_motion_df[('Right Foot Middle', 'y')].values,
    cleaned_motion_df[('Left Foot Middle', 'Vel_x')].values,
    cleaned_motion_df[('Left Foot Middle', 'Vel_y')].values,
    cleaned_motion_df[('Right Foot Middle', 'Vel_x')].values,
    cleaned_motion_df[('Right Foot Middle', 'Vel_y')].values,
    cleaned_motion_df[('Left Foot Middle', 'Acc_x')].values,
    cleaned_motion_df[('Left Foot Middle', 'Acc_y')].values,
    cleaned_motion_df[('Right Foot Middle', 'Acc_x')].values,
    cleaned_motion_df[('Right Foot Middle', 'Acc_y')].values
])

predictions = model.predict(features)

cleaned_motion_df['GRF_L_x'] = predictions[:,0]
cleaned_motion_df['GRF_L_y'] = predictions[:,1]
cleaned_motion_df['GRF_R_x'] = predictions[:,2]
cleaned_motion_df['GRF_R_y'] = predictions[:,3]


In [33]:
cleaned_motion_df.head(10)

Left Hip                           Right Hip                          \
                x           y       Angle           x           y       Angle   
Frame                                                                           
0      484.622067  516.190932  172.722274  386.081599  521.689804  173.772676   
1      484.232980  515.581899  172.764216  385.352599  520.257279  174.035576   
2      484.190542  514.966686  172.851308  385.114314  519.018935  174.115627   
3      484.195467  514.734814  172.930039  385.114387  518.709784  174.141637   
4      484.202129  514.752452  172.919586  385.130681  518.733505  174.218508   
5      484.228007  514.934210  172.919814  385.352671  518.934445  174.201647   
6      484.383178  515.073416  172.915554  385.623955  519.051572  174.225127   
7      484.404373  514.191055  172.897439  385.747455  518.446047  174.337111   
8      484.076747  513.266047  172.722201  385.370632  517.564201  174.604522   
9      483.846066  512.591676  172.512534  384.536623  517.467545  175.101510   

        Left Knee                          Right Knee  ... Left Foot Middle  \
                x           y       Angle           x  ...            Acc_x   
Frame                                                  ...                    
0      500.704007  684.669213  178.475596  392.666211  ...         0.000000   
1      499.247358  684.077175  179.169102  392.671714  ...         0.000000   
2      498.423319  683.551826  179.572981  392.672463  ...         0.071357   
3      497.993581  683.450761  179.805143  392.729119  ...         0.108967   
4      497.956020  683.432415  179.872417  392.996999  ...         0.040145   
5      497.940860  683.427844  179.887606  393.157892  ...        -0.026844   
6      497.923576  683.412781  179.958349  393.352990  ...        -0.007652   
7      497.827644  682.528102  179.951600  393.865214  ...        -0.037658   
8      497.777481  682.025220  179.960905  394.212925  ...         0.009632   
9      497.785544  680.107677  179.932781  394.851956  ...        -0.093349   

                Right Foot Middle                                 GRF_L_x  \
          Acc_y             Vel_x     Vel_y     Acc_x     Acc_y             
Frame                                                                       
0      0.000000          0.000000  0.000000  0.000000  0.000000  0.001271   
1      0.000000          0.026143  1.383247  0.000000  0.000000 -0.010320   
2      0.001835         -0.014617  0.747854 -0.040760 -0.635394 -0.005930   
3      0.019730         -0.076680  0.432780 -0.062064 -0.315074  0.001847   
4      0.059030         -0.048871  0.245551  0.027809 -0.187229  0.004653   
5     -0.113779         -0.100784 -0.030352 -0.051913 -0.275903  0.003518   
6      0.010267          0.084779 -0.029901  0.185563  0.000451 -0.002790   
7      0.001674          0.279781 -0.008755  0.195002  0.021147 -0.001207   
8     -0.056809          0.099010  0.016061 -0.180771  0.024816 -0.002704   
9     -0.174226          0.189667  0.053526  0.090657  0.037465 -0.005949   

        GRF_L_y   GRF_R_x   GRF_R_y  
                                     
Frame                                
0      0.075597 -0.017432 -0.112449  
1      0.069719 -0.015125 -0.103248  
2      0.072157 -0.019284 -0.102438  
3      0.083290 -0.020734 -0.096390  
4      0.069748 -0.021680 -0.116859  
5      0.077110 -0.021981 -0.130282  
6      0.088004 -0.021086 -0.140557  
7      0.097338 -0.018796 -0.103172  
8      0.091542 -0.023736 -0.102709  
9      0.091429 -0.015042 -0.061678  

[10 rows x 40 columns]

In [34]:
#Calculate Joint Moments

####NEEDS USER INPUT##### TODO: ADD USER INPUT
body_mass = 70 #KG
shank_mass = 0.0457 * body_mass
thigh_mass = 0.1447 * body_mass
foot_mass = 0.0133 * body_mass
g = 9.81 

shank_length = np.linalg.norm(np.array([cleaned_motion_df[('Left Knee', 'x')], cleaned_motion_df[('Left Knee', 'y')]]) - np.array([cleaned_motion_df[('Left Ankle', 'x')], cleaned_motion_df[('Left Ankle', 'y')]]))
thigh_length = np.linalg.norm(np.array([cleaned_motion_df[('Left Hip', 'x')], cleaned_motion_df[('Left Hip', 'y')]]) - np.array([cleaned_motion_df[('Left Knee', 'x')], cleaned_motion_df[('Left Knee', 'y')]]))
foot_length = np.linalg.norm(np.array([cleaned_motion_df[('Left Ankle', 'x')], cleaned_motion_df[('Left Ankle', 'y')]]) - np.array([cleaned_motion_df[('Left Foot Middle', 'x')], cleaned_motion_df[('Left Foot Middle', 'y')]]))

I_foot = 1/3 * foot_mass * foot_length**2
I_shank = 1/3 * shank_mass * shank_length**2
I_thigh = 1/3 * thigh_mass * thigh_length**2




net_moments = []


for index, row in cleaned_motion_df.iterrows():

    left_hip    = np.array([row[('Left Hip', 'x')], row[('Left Hip', 'y')]])
    left_knee   = np.array([row[('Left Knee', 'x')], row[('Left Knee', 'y')]])
    left_ankle  = np.array([row[('Left Ankle', 'x')], row[('Left Ankle', 'y')]])
    left_foot   = np.array([row[('Left Foot Middle', 'x')], row[('Left Foot Middle', 'y')]])

    right_hip    = np.array([row[('Right Hip', 'x')], row[('Right Hip', 'y')]])
    right_knee   = np.array([row[('Right Knee', 'x')], row[('Right Knee', 'y')]])
    right_ankle  = np.array([row[('Right Ankle', 'x')], row[('Right Ankle', 'y')]])
    right_foot   = np.array([row[('Right Foot Middle', 'x')], row[('Right Foot Middle', 'y')]])
    
    left_shank_CoM = (left_knee + left_ankle) / 2
    left_thigh_CoM = (left_hip + left_knee) / 2
    

    right_shank_CoM = (right_knee + right_ankle) / 2
    right_thigh_CoM = (right_hip + right_knee) / 2
    

    F_foot = np.array([row['GRF_L_x'], row['GRF_L_y']])
    
    
    alpha_ankle = row[('Left Ankle', 'Angular Acceleration')]
    moment_ankle_inertial = I_foot * alpha_ankle

    r_ankle = left_foot - left_ankle
    moment_ankle = r_ankle[0]*F_foot[1] - r_ankle[1]*F_foot[0]
    

    r_knee = left_ankle - left_knee

    moment_knee_external = r_knee[0]*F_foot[1] - r_knee[1]*F_foot[0]

    alpha_shank = row[('Left Knee', 'Angular Acceleration')]
    moment_knee_inertial = I_shank * alpha_shank

    r_hip = left_knee - left_hip
    moment_hip_external = r_hip[0]*F_foot[1] - r_hip[1]*F_foot[0]

    alpha_thigh = row[('Left Hip', 'Angular Acceleration')]
    moment_hip_inertial = I_thigh * alpha_thigh + I_shank * alpha_shank
    
    
    r_gravity_shank = left_knee - left_shank_CoM
    r_gravity_thigh = left_hip - left_thigh_CoM

    F_gravity_shank = np.array([0, -shank_mass * g])
    F_gravity_thigh = np.array([0, -thigh_mass * g])

    moment_gravity_shank = r_gravity_shank[0]*F_gravity_shank[1] - r_gravity_shank[1]*F_gravity_shank[0]
    moment_gravity_thigh = r_gravity_thigh[0]*F_gravity_thigh[1] - r_gravity_thigh[1]*F_gravity_thigh[0]

 
    net_moment_left_ankle = moment_ankle  - moment_ankle_inertial
    net_moment_left_knee  = moment_knee_external  - moment_knee_inertial - moment_gravity_shank
    net_moment_left_hip   = moment_hip_external  - moment_hip_inertial  - moment_gravity_thigh


    F_foot = np.array([row['GRF_R_x'], row['GRF_R_y']])
    
    
    alpha_ankle = row[('Right Ankle', 'Angular Acceleration')]
    moment_ankle_inertial = I_foot * alpha_ankle

    r_ankle = right_foot - right_ankle
    moment_ankle = r_ankle[0]*F_foot[1] - r_ankle[1]*F_foot[0]
    

    r_knee = right_ankle - right_knee

    moment_knee_external = r_knee[0]*F_foot[1] - r_knee[1]*F_foot[0]

    alpha_shank = row[('Right Knee', 'Angular Acceleration')]
    moment_knee_inertial = I_shank * alpha_shank

    r_hip = right_knee - right_hip
    moment_hip_external = r_hip[0]*F_foot[1] - r_hip[1]*F_foot[0]

    alpha_thigh = row[('Right Hip', 'Angular Acceleration')]
    moment_hip_inertial = I_thigh * alpha_thigh + I_shank * alpha_shank
    
    
    r_gravity_shank = right_knee - right_shank_CoM
    r_gravity_thigh = right_hip - right_thigh_CoM

    F_gravity_shank = np.array([0, -shank_mass * g])
    F_gravity_thigh = np.array([0, -thigh_mass * g])

    moment_gravity_shank = r_gravity_shank[0]*F_gravity_shank[1] - r_gravity_shank[1]*F_gravity_shank[0]
    moment_gravity_thigh = r_gravity_thigh[0]*F_gravity_thigh[1] - r_gravity_thigh[1]*F_gravity_thigh[0]

 
    net_moment_right_ankle = moment_ankle  - moment_ankle_inertial
    net_moment_right_knee  = moment_knee_external  - moment_knee_inertial - moment_gravity_shank
    net_moment_right_hip   = moment_hip_external  - moment_hip_inertial  - moment_gravity_thigh

    net_moments.append({
        'Frame': index,
        'Left_Ankle_Moment': net_moment_left_ankle,
        'Left_Knee_Moment': net_moment_left_knee,
        'Left_Hip_Moment': net_moment_left_hip,
        'Right_Ankle_Moment': net_moment_right_ankle,
        'Right_Knee_Moment': net_moment_right_knee,
        'Right_Hip_Moment': net_moment_right_hip
    })

moments_df = pd.DataFrame(net_moments)

aggregate_moment = moments_df.sum().abs()





In [35]:
aggregate_moment

Frame                                42486
Left_Ankle_Moment     [27866.023876313062]
Left_Knee_Moment      [112030.13251608257]
Left_Hip_Moment        [1090266.241407865]
Right_Ankle_Moment    [22509.496299490194]
Right_Knee_Moment      [35899.52207808211]
Right_Hip_Moment       [1321122.002591573]
dtype: object